   user기반
-  movie
-  user - 10,000
-  item(movie) - 200개 영화를 평가
-  (한 유저의 여러개 아이템)
-  (user가 0이 많을 것)
-  (user가 더 많으니까 user를 filtering) 
----------------------------------------

   item기반
-  user - 10,000
-  itme(product) - 100,000개 상품 보유
-  (한 유저가 적은 수의 아이템)
-  (item이 0이 많을 것)
-  (item가 더 많으니까 item을 filtering)

### Euclidean Distance Similarity - 유클리드 거리 유사도

In [7]:
import numpy as np

In [8]:
# 샘플 데이터
vector_1 = np.array([1,2,3,4,5]) 
vector_2 = np.array([5,4,3,2,1]) 
vector_3 = np.array([2,3,4,5,6])

In [6]:
print((vector_1 - vector_2))
print((vector_1 - vector_2) ** 2)
print(sum((vector_1 - vector_2) ** 2))
print(np.sqrt(sum((vector_1 - vector_2) ** 2)))

[-4 -2  0  2  4]
[16  4  0  4 16]
40
6.32455532034


In [11]:
np.linalg.norm(vector_1 - vector_2),\
np.linalg.norm(vector_1 - vector_3),\
np.linalg.norm(vector_2 - vector_3)

(6.324555320336759, 2.2360679774997898, 6.7082039324993694)

### Cosine Similarity - 코사인 유사도

In [12]:
import numpy as np
from scipy import spatial

In [13]:
vector_1 = np.array([1,2,3,4,5])
vector_2 = np.array([5,4,3,2,1])
vector_3 = np.array([11,19,28,32,47])

In [14]:
# 분자 - 벡터의 내적
print(sum(vector_1 * vector_2))
print(np.dot(vector_1, vector_2)) # 내적 numpy 함수

35
35


In [15]:
# 분모 - 벡터의 크기
print(vector_1 * vector_1, vector_2 * vector_2)
print(sum(vector_1 * vector_1), sum(vector_2 * vector_2))
print(np.sqrt(sum(vector_1 * vector_1)), np.sqrt(sum(vector_2 * vector_2)))

[ 1  4  9 16 25] [25 16  9  4  1]
55 55
7.4161984871 7.4161984871


In [16]:
print(np.sqrt(np.dot(vector_1, vector_1)) * np.sqrt(np.dot(vector_2, vector_2)))

55.0


In [17]:
# scipy 함수 사용
spatial.distance.cosine(vector_1, vector_2), 1 - spatial.distance.cosine(vector_1, vector_3)

(0.36363636363636365, 0.99710830870489031)

In [18]:
import numpy as np
import pandas as pd
from scipy import spatial

# sample data set matrix
columns = ["article_1","article_2","article_3","article_4","article_5"]
index = ["user_1", "user_2", "user_3", "user_4"]
data = np.array([ 
    [5,3,0,0,2], 
    [2,0,0,1,4], 
    [0,0,4,3,1], 
    [4,0,4,5,0],
])

sample_df = pd.DataFrame(data, columns=columns, index=index)
sample_df


,article_1,article_2,article_3,article_4,article_5
user_1,5,3,0,0,2
user_2,2,0,0,1,4
user_3,0,0,4,3,1
user_4,4,0,4,5,0


In [19]:
def cosine_smimilarity(vector_1, vector_2):
    # vector_1 데이터가 0인 index를 제거
    idx = vector_1.nonzero()[0] 
    # vector에서 value가 0이 아닌 index를 구함 # index 값으로 vector의 요소를 필터링 함
    vector_1, vector_2 = np.array(vector_1)[idx], np.array(vector_2)[idx]
    return 1 - spatial.distance.cosine(vector_1, vector_2)

In [20]:
cosine_smimilarity(sample_df.loc["user_1"], sample_df.loc["user_2"])

0.65292862509901051

In [23]:
# 유사도 행렬 함수
def similarity_matrix(sample_df, similarity_func):
    
    # index 데이터 저장
    index = sample_df.index
    
    # 데이터 프레임 전치 (index - article, columns - user) 
    df = sample_df.T
    
    # 모든 user 데이터 사이의 유사도를 구해 행렬 생성 
    matrix = []
    
    for idx_1, value_1 in df.items(): 
        # row 데이터 저장
        row = []
        for idx_2, value_2 in df.items():
            # 두 user 사이의 유사도 구함
            row.append(similarity_func(value_1, value_2)) 
        matrix.append(row)
        
    return pd.DataFrame(matrix, columns=index, index=index)

sm_df = similarity_matrix(sample_df, cosine_smimilarity)
sm_df

,user_1,user_2,user_3,user_4
user_1,1.000000,0.652929,0.324443,0.811107
user_2,0.729397,1.000000,0.483046,0.443039
user_3,0.196116,0.332956,1.000000,0.949474
user_4,0.529813,0.770054,0.821210,1.000000


In [29]:
# 추천할 대상 및 추천 대상과 유사한 몇개의 데이터까지 사용할지에 대해 설정
user, closer_count = "user_1", 2

# 본인 데이터 제거
ms_df = sm_df.drop(user)

# 유사도가 높은 순으로 sorting
ms_df = ms_df.sort_values(user, ascending=False)

# 위의 설정 대로 컨텐츠를 추천할 사용자와 유사도가 높은 사용자 필터링
ms_df = ms_df[:closer_count]

In [31]:
# use_1과 가까운 상위 2개 데이터
sample_df.loc[ms_df.index]

,article_1,article_2,article_3,article_4,article_5
user_2,2,0,0,1,4
user_4,4,0,4,5,0


In [33]:
mean = np.zeros(len(sample_df.columns))
for ms_user, sms_value in ms_df[user].items():
    mean += sample_df.loc[ms_user] 
mean /= len(ms_df[user])
    
pred_df = pd.DataFrame(columns=sample_df.columns) 
pred_df.loc["user"] = sample_df.loc[user] 
pred_df.loc["mean"] = mean
pred_df

,article_1,article_2,article_3,article_4,article_5
user,5,3,0,0,2
mean,3,0,2,3,2


In [34]:
def mean_score(sample_df, sm_df, target, closer_count):
    # 유사도 행렬에서 추천 user와 가까운 user의 유사도 데이터 프레임 
    ms_df = sm_df.drop(target)
    ms_df = ms_df.sort_values(target, ascending=False) 
    ms_df = ms_df[target][:closer_count]
    
    # 유사도가 높은 user를 나타내는 데이터 프레임 
    ms_df = sample_df.loc[ms_df.index]
    
    # 결과 데이터 프레임 생성
    pred_df = pd.DataFrame(columns=sample_df.columns) 
    pred_df.loc["user"] = sample_df.loc[target] 
    pred_df.loc["mean"] = ms_df.mean()
    
    return pred_df

In [35]:
target, closer_count = "user_1", 2
pred_df = mean_score(sample_df, sm_df, target, closer_count)
pred_df

,article_1,article_2,article_3,article_4,article_5
user,5,3,0,0,2
mean,3,0,2,3,2


In [36]:
# user가 읽지 않은 기사를 순서대로 나열 추천 기사 정렬 및 출력 
recommand_df = pred_df.T
recommand_df = recommand_df[recommand_df["user"] == 0] 
recommand_df = recommand_df.sort_values("mean", ascending=False) 
print(list(recommand_df.index))
recommand_df

['article_4', 'article_3']


,user,mean
article_4,0,3
article_3,0,2


In [38]:
def mse(value, pred):
    # user 데이터에서 0인 데이터 제거 
    idx = value.nonzero()[0]
    value, pred = np.array(value)[idx], np.array(pred)[idx]
    # 수식 계산후 결과 리턴
    return sum((value - pred)**2) / len(idx)

mse(pred_df.loc["user"], pred_df.loc["mean"])

4.333333333333333

In [39]:
# 전체 user에 대한 평가
def evaluate(df, sm_df, closer_count, algorithm):
    # user 리스트 
    users = df.index 
    evaluate_list = []
    # 모든 user에 대해서 mae 값을 구함 
    for target in users:
        result_df = mean_score(df, sm_df, target, closer_count) 
        evaluate_list.append(algorithm(result_df.loc["user"], result_df.loc["mean"]))
    # 모든 user의 mae값의 평균을 리턴 
    return np.average(evaluate_list)

evaluate(sample_df, sm_df, 2, mse)

4.5

In [40]:
def rmse(value, pred):
    
    # user 데이터에서 0인 데이터 제거 
    idx = value.nonzero()[0]
    value, pred = np.array(value)[idx], np.array(pred)[idx]
    
    # 수식 계산후 결과 리턴
    return np.sqrt(sum((value - pred)**2) / len(idx))

rmse(pred_df.loc["user"], pred_df.loc["mean"])

2.0816659994661326

In [41]:
evaluate(sample_df, sm_df, closer_count, rmse)

2.0677918275480169

In [45]:
# 한명의 user에 대한 MAE 값
def mae(value, pred):
    
    # user 데이터에서 0인 데이터 제거
    idx = value.nonzero()[0]
    value, pred = np.array(value)[idx], np.array(pred)[idx]
    
    # 수식 계산후 결과 리턴
    return np.absolute(sum(value - pred)) / len(idx)

mae(pred_df.loc["user"], pred_df.loc["mean"])

1.6666666666666667

In [47]:
evaluate(sample_df, sm_df, closer_count, mae)

1.1666666666666667

In [48]:
# Sample Dataset
sample_df

,article_1,article_2,article_3,article_4,article_5
user_1,5,3,0,0,2
user_2,2,0,0,1,4
user_3,0,0,4,3,1
user_4,4,0,4,5,0


In [49]:
# Similarity Matrix
sm_df

,user_1,user_2,user_3,user_4
user_1,1.000000,0.652929,0.324443,0.811107
user_2,0.729397,1.000000,0.483046,0.443039
user_3,0.196116,0.332956,1.000000,0.949474
user_4,0.529813,0.770054,0.821210,1.000000


In [52]:
# Closer User Dataset (for user_1)
sample_df.loc[ms_df.index]

,article_1,article_2,article_3,article_4,article_5
user_2,2,0,0,1,4
user_4,4,0,4,5,0


In [53]:
# Prediction Dataset (for user_1)
pred_df

,article_1,article_2,article_3,article_4,article_5
user,5,3,0,0,2
mean,3,0,2,3,2
